Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST_clean.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8913, 28, 28) (8913,)
Test set (8695, 28, 28) (8695,)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8913, 784) (8913, 10)
Test set (8695, 784) (8695, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [51]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  beta = tf.Variable(tf.zeros([1]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)
    )
  # The 5e-4 can be chosen via tuning, not through optimisation
  loss += 5e-4 * tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [52]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.284346
Training accuracy: 9.2%
Validation accuracy: 12.3%
Loss at step 100: 3.561872
Training accuracy: 73.0%
Validation accuracy: 69.6%
Loss at step 200: 2.986425
Training accuracy: 76.2%
Validation accuracy: 71.9%
Loss at step 300: 2.632870
Training accuracy: 77.5%
Validation accuracy: 72.6%
Loss at step 400: 2.372541
Training accuracy: 78.4%
Validation accuracy: 73.3%
Loss at step 500: 2.166132
Training accuracy: 79.0%
Validation accuracy: 73.6%
Loss at step 600: 1.995484
Training accuracy: 79.7%
Validation accuracy: 73.9%
Loss at step 700: 1.850627
Training accuracy: 80.3%
Validation accuracy: 74.0%
Loss at step 800: 1.725360
Training accuracy: 81.1%
Validation accuracy: 74.4%
Test accuracy: 82.0%


## Let's do it with a NN now

In [18]:
batch_size = 128
n_hidden_1 = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  



  weights_h1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1]), 'w_in')
  biases_h1 = tf.Variable(tf.truncated_normal([n_hidden_1]), 'b_in')
      
  weights_out = tf.Variable(tf.truncated_normal([n_hidden_1, num_labels]), 'w_o')
  biases_out = tf.Variable(tf.zeros([num_labels]), 'b_o')
    
  layer_1 = tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_h1), biases_h1))
  out_layer = tf.matmul(layer_1, weights_out) + biases_out
    

    
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels))
 
  loss += 5e-4 * tf.nn.l2_loss(weights_h1)
  loss += 5e-4 * tf.nn.l2_loss(weights_out)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  


  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out_layer)
  
  valid_prediction = tf.nn.softmax(
            tf.matmul(
                tf.nn.relu(
                    tf.add(
                        tf.matmul(tf_valid_dataset, weights_h1),
                        biases_h1)),
                    weights_out) + biases_out
            )
    
  test_prediction = tf.nn.softmax(
            tf.matmul(
                tf.nn.relu(
                    tf.add(
                        tf.matmul(tf_test_dataset, weights_h1),
                        biases_h1)
                    ), weights_out) + biases_out)

In [57]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 656.875122
Minibatch accuracy: 13.3%
Validation accuracy: 30.3%
Minibatch loss at step 500: 125.473610
Minibatch accuracy: 75.8%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 97.064384
Minibatch accuracy: 83.6%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 73.731941
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 57.289448
Minibatch accuracy: 84.4%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 43.705448
Minibatch accuracy: 89.1%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 34.467216
Minibatch accuracy: 88.3%
Validation accuracy: 83.4%
Test accuracy: 90.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [21]:
num_steps = 3001
max_batches = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    step_limited = step % max_batches
    offset = (step_limited * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 631.970093
Minibatch accuracy: 11.7%
Validation accuracy: 37.1%
Minibatch loss at step 500: 122.082016
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 1000: 95.019913
Minibatch accuracy: 100.0%
Validation accuracy: 77.3%
Minibatch loss at step 1500: 73.985336
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 2000: 57.612309
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
Minibatch loss at step 2500: 44.863998
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Minibatch loss at step 3000: 34.937878
Minibatch accuracy: 100.0%
Validation accuracy: 76.9%
Test accuracy: 83.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [22]:
batch_size = 128
n_hidden_1 = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  



  weights_h1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_1]), 'w_in')
  biases_h1 = tf.Variable(tf.truncated_normal([n_hidden_1]), 'b_in')
      
  weights_out = tf.Variable(tf.truncated_normal([n_hidden_1, num_labels]), 'w_o')
  biases_out = tf.Variable(tf.zeros([num_labels]), 'b_o')
    
  layer_1 = tf.nn.dropout(
                tf.nn.relu(tf.add(tf.matmul(tf_train_dataset, weights_h1), biases_h1)),
                0.5)
  out_layer = tf.matmul(layer_1, weights_out) + biases_out
    

    
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(out_layer, tf_train_labels))
 
  loss += 5e-4 * tf.nn.l2_loss(weights_h1)
  loss += 5e-4 * tf.nn.l2_loss(weights_out)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  


  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(out_layer)
  
  valid_prediction = tf.nn.softmax(
            tf.matmul(
                tf.nn.relu(
                    tf.add(
                        tf.matmul(tf_valid_dataset, weights_h1),
                        biases_h1)),
                    weights_out) + biases_out
            )
    
  test_prediction = tf.nn.softmax(
            tf.matmul(
                tf.nn.relu(
                    tf.add(
                        tf.matmul(tf_test_dataset, weights_h1),
                        biases_h1)
                    ), weights_out) + biases_out)

In [24]:
num_steps = 3001
max_batches = 100

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    step_limited = step % max_batches
    offset = (step_limited * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 668.178406
Minibatch accuracy: 6.2%
Validation accuracy: 26.2%
Minibatch loss at step 500: 139.365067
Minibatch accuracy: 78.9%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 98.443062
Minibatch accuracy: 85.9%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 76.944496
Minibatch accuracy: 89.1%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 59.647297
Minibatch accuracy: 85.2%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 45.970875
Minibatch accuracy: 93.0%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 35.796268
Minibatch accuracy: 93.0%
Validation accuracy: 82.1%
Test accuracy: 89.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [73]:
batch_size = 128
n_hidden_1 = 1024

def new_wb(dimensions):
    weights = tf.Variable(tf.truncated_normal(dimensions))
    biases = tf.Variable(tf.truncated_normal(dimensions[1:]))
    return weights, biases

def create_weights(dimensions):
    result = []
    for i in range(len(dimensions) -1):
        result.append(new_wb(dimensions[i:i+2]))
    return result

def create_nn(tensor_input, weights, layers):
    for i in range(len(layers)):
        
        # Get the weights and biases from the array
        w, b = weights[i]
        
        # (x * weights + biases)
        new_layer = tf.add(tf.matmul(tensor_input, w), b) 
        
        # Run the modifiers, like relu...
        modifiers = layers[i]
        if modifiers is not null:
            if len(modifiers) <> len(layers):
                raise Exception("Weights and layers should be the same size")
            for modifier in modifiers:
                new_layer = modifier(new_layer)
            
        tensor_input = new_layer

    return tensor_input
        
    

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
        
           
  weights = create_weights([image_size * image_size, n_hidden_1, num_labels])
  
  layers_train = [
      [tf.nn.relu, lambda x: tf.nn.dropout(x, 0.5)],
      [],
  ]
  #layers_train = [[tf.nn.relu, lambda x: tf.nn.dropout(x, 0.5)]]

  train_nn = create_nn(tf_train_dataset, weights, layers_train)

  # Loss function with l2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(train_nn, tf_train_labels)) 
  # Add l2 loss
  for w, b in weights:
        loss += 5e-4 * tf.nn.l2_loss(w) # + 5e-4 * tf.nn.l2_loss(b)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.25).minimize(loss)


  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(train_nn)

  layers_test_validation = [
      [tf.nn.relu],
      [],
  ]
  #layers_test_validation = [[]]
  
  valid_prediction = tf.nn.softmax(create_nn(tf_valid_dataset, weights, layers_test_validation))
  test_prediction = tf.nn.softmax(create_nn(tf_test_dataset, weights, layers_test_validation))

In [72]:
num_steps = 3001
max_batches = 100000
    
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    step_limited = step % max_batches
    offset = (step_limited * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]    
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 469.163391
Minibatch accuracy: 10.2%
Validation accuracy: 32.5%
Minibatch loss at step 500: 148.690460
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 126.492523
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 111.492783
Minibatch accuracy: 85.2%
Validation accuracy: 82.3%
Minibatch loss at step 2000: 94.527817
Minibatch accuracy: 89.1%
Validation accuracy: 82.5%
Minibatch loss at step 2500: 83.104202
Minibatch accuracy: 89.8%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 72.814865
Minibatch accuracy: 85.9%
Validation accuracy: 82.4%
Test accuracy: 89.0%
